<a href="https://colab.research.google.com/github/ArunK-ML/Project---Live-PM2.5-Nowcast-and-Forecast---Final-Project/blob/main/PM_2_0_BEST_APPROCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import joblib
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone
from sklearn.ensemble import IsolationForest, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# ==========================================================
# CONFIG
# ==========================================================
TZ = timezone.utc
# Default location (Chennai). Change to desired lat/lon.
LAT, LON = 13.0827, 80.2707
HOURS_BACK = 72
RADIUS_KM = 30

# ==========================================================
# 1️⃣ DATA FETCHING
# ==========================================================

def fetch_openaq_pm25(lat: float, lon: float, hours: int = 168, radius_km: int = 30) -> pd.DataFrame:
    """Fetch PM2.5 measurements from OpenAQ and return hourly median with a 'timestamp' column (UTC)."""
    end = datetime.now(TZ)
    start = end - timedelta(hours=hours)
    base = "https://api.openaq.org/v2/measurements"
    params = {
        "coordinates": f"{lat},{lon}",
        "radius": int(radius_km * 1000),
        "parameter": "pm25",
        "date_from": start.isoformat(),
        "date_to": end.isoformat(),
        "limit": 10000,
        "sort": "desc",
        "order_by": "datetime",
        "page": 1,
    }
    frames = []
    try:
        while True:
            r = requests.get(base, params=params, timeout=30)
            if r.status_code >= 400:
                break
            js = r.json()
            items = js.get("results", [])
            if not items:
                break
            df = pd.DataFrame(items)
            if "date" not in df:
                break
            # extract UTC datetime
            df["timestamp"] = pd.to_datetime(df["date"].apply(lambda d: d.get("utc")), utc=True)
            df = df[["timestamp", "value"]].rename(columns={"value": "pm25"})
            frames.append(df)
            meta = js.get("meta", {})
            found = meta.get("found")
            page = params["page"]
            limit = params["limit"]
            if found is None or page * limit >= int(found):
                break
            params["page"] = page + 1
    except Exception:
        frames = []

    if not frames:
        return pd.DataFrame(columns=["timestamp", "pm25"])

    df = pd.concat(frames, ignore_index=True)
    df = df[(df["timestamp"] >= start) & (df["timestamp"] <= end)]

    # hourly median across stations
    df_hour = (
        df.set_index("timestamp")
          .groupby(pd.Grouper(freq="1H"))["pm25"]
          .median()
          .reset_index()
    )
    # ensure sorted
    df_hour = df_hour.sort_values("timestamp").reset_index(drop=True)
    return df_hour

def fetch_openmeteo_aq_pm25(lat: float, lon: float, hours: int = 168) -> pd.DataFrame:
    """Fallback: open-meteo air-quality PM2.5 hourly. Returns 'timestamp' and 'pm25' columns (UTC)."""
    end = datetime.now(TZ)
    start = end - timedelta(hours=hours)
    # past_days param uses days; request a few days to cover hours window
    past_days = max(1, (hours // 24) + 1)
    url = (
        "https://air-quality-api.open-meteo.com/v1/air-quality?"
        f"latitude={lat}&longitude={lon}&hourly=pm2_5&past_days={past_days}&timezone=UTC"
    )
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        h = r.json().get("hourly", {})
        if not h:
            return pd.DataFrame(columns=["timestamp", "pm25"])
        df = pd.DataFrame({"timestamp": h.get("time", []), "pm25": h.get("pm2_5", [])})
        df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
        df = df[(df["timestamp"] >= start) & (df["timestamp"] <= end)].reset_index(drop=True)
        return df
    except Exception:
        return pd.DataFrame(columns=["timestamp", "pm25"])

def fetch_openmeteo_weather(lat: float, lon: float, hours: int = 168) -> pd.DataFrame:
    """Fetch meteorological hourly features from Open-Meteo (temperature, humidity, windspeed)."""
    end = datetime.now(TZ)
    start = end - timedelta(hours=hours)
    past_days = max(1, (hours // 24) + 1)
    url = (
        "https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}&hourly=temperature_2m,relativehumidity_2m,windspeed_10m"
        f"&past_days={past_days}&timezone=UTC"
    )
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        j = r.json().get("hourly", {})
        if not j:
            return pd.DataFrame(columns=["timestamp"])
        df = pd.DataFrame({
            "timestamp": j.get("time", []),
            "temperature_2m": j.get("temperature_2m", []),
            "relativehumidity_2m": j.get("relativehumidity_2m", []),
            "windspeed_10m": j.get("windspeed_10m", []),
        })
        df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
        return df[(df["timestamp"] >= start) & (df["timestamp"] <= end)].reset_index(drop=True)
    except Exception:
        return pd.DataFrame(columns=["timestamp"])

# ==========================================================
# 2️⃣ MERGE + PREPROCESSING
# ==========================================================

def merge_datasets(pm_df: pd.DataFrame, weather_df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge PM2.5 and weather data by timestamp (asof-merge).
    If weather_df is empty, returns pm_df with timestamp/pm25 only.
    """
    print("🔗 Merging datasets...")
    pm = pm_df.copy()
    pm = pm.sort_values("timestamp").reset_index(drop=True)
    if weather_df is None or weather_df.empty:
        return pm
    w = weather_df.copy().sort_values("timestamp").reset_index(drop=True)
    # asof merge: nearest earlier weather observation for each pm timestamp
    merged = pd.merge_asof(pm, w, on="timestamp", direction="nearest", tolerance=pd.Timedelta("1H"))
    # if some weather cols are missing due to tolerance, that's okay
    return merged

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """Clean and preprocess merged dataset. Ensures hourly frequency, interpolates missing values."""
    print("🧹 Cleaning data...")
    if df.empty:
        return df
    df = df.copy()
    # ensure timestamp is datetime tz-aware
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
    df = df.drop_duplicates(subset=["timestamp"])
    df = df.set_index("timestamp").sort_index()
    # resample to 1H and take mean for numeric columns
    df = df.resample("1H").mean()
    # interpolate small gaps, then forward/backward fill remaining
    df = df.interpolate(limit=3).ffill().bfill()
    df = df.reset_index()
    return df

def remove_outliers(df: pd.DataFrame, cols=None, z_thresh=3) -> pd.DataFrame:
    """Remove statistical outliers by z-score (replace with NaN -> interpolate)."""
    print("🚫 Removing outliers...")
    if df.empty:
        return df
    df = df.copy()
    if cols is None:
        cols = [c for c in df.columns if c not in ("timestamp",)]
    for c in cols:
        if c not in df.columns or not np.issubdtype(df[c].dtype, np.number):
            continue
        std = df[c].std()
        mean = df[c].mean()
        if std == 0 or np.isnan(std):
            continue
        z = np.abs((df[c] - mean) / std)
        df.loc[z > z_thresh, c] = np.nan
    # interpolate and fill
    df = df.interpolate(limit=6).ffill().bfill()
    return df

# ==========================================================
# 3️⃣ FEATURE ENGINEERING
# ==========================================================

def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["hour"] = df["timestamp"].dt.hour
    df["dayofweek"] = df["timestamp"].dt.dayofweek
    return df

def add_lag_features(df: pd.DataFrame, col="pm25", lags=6) -> pd.DataFrame:
    df = df.copy()
    for i in range(1, lags + 1):
        df[f"{col}_lag_{i}"] = df[col].shift(i)
    return df

def add_rolling_features(df: pd.DataFrame, col="pm25", windows=[3,6,12]) -> pd.DataFrame:
    df = df.copy()
    for w in windows:
        df[f"{col}_roll_mean_{w}"] = df[col].rolling(w, min_periods=1).mean()
        df[f"{col}_roll_std_{w}"] = df[col].rolling(w, min_periods=1).std().fillna(0)
    return df

def feature_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    print("🧠 Creating time, lag, and rolling features...")
    df = df.copy()
    # ensure pm25 exists
    if "pm25" not in df.columns:
        raise KeyError("pm25 column not found in dataframe (needed for feature engineering).")
    df = add_time_features(df)
    df = add_lag_features(df)
    df = add_rolling_features(df)
    # drop rows with NaNs introduced by lagging
    df = df.dropna().reset_index(drop=True)
    return df

# ==========================================================
# 4️⃣ ANOMALY DETECTION
# ==========================================================

def detect_anomalies(df: pd.DataFrame, col="pm25") -> pd.DataFrame:
    print("⚠️ Detecting anomalies (IsolationForest)...")
    df = df.copy()
    if col not in df.columns or df.empty:
        df["anomaly_flag"] = 0
        return df
    # IsolationForest expects 2D numeric input and no NaNs
    X = df[[col]].fillna(method="ffill").fillna(method="bfill")
    model = IsolationForest(contamination=0.05, random_state=42)
    flags = model.fit_predict(X)
    df["anomaly_flag"] = np.where(flags == -1, 1, 0)
    return df

# ==========================================================
# 5️⃣ TRAINING
# ==========================================================

def train_model(df: pd.DataFrame, model_out="models/model.joblib"):
    """Train Gradient Boosting model."""
    os.makedirs(os.path.dirname(model_out), exist_ok=True)

    df = df.copy()
    df = feature_pipeline(df)

    # drop non-feature columns
    drop_cols = ["timestamp", "anomaly_flag"]
    X = df.drop(columns=[c for c in drop_cols if c in df.columns] + ["pm25"], errors="ignore")
    y = df["pm25"]

    # simple chronological split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

    print("🚀 Training model (Gradient Boosting)...")
    model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_val)
    mae = mean_absolute_error(y_val, preds)
    print(f"✅ Validation MAE: {mae:.3f}")

    joblib.dump(model, model_out)
    print(f"💾 Model saved to {model_out}")

    return model, mae

# ==========================================================
# 6️⃣ MAIN PIPELINE
# ==========================================================

def main():
    os.makedirs("data/processed", exist_ok=True)
    os.makedirs("models", exist_ok=True)

    # Fetch PM2.5 (OpenAQ preferred, fallback open-meteo)
    pm_df = fetch_openaq_pm25(LAT, LON, hours=HOURS_BACK, radius_km=RADIUS_KM)
    if pm_df.empty:
        print("No OpenAQ data found — falling back to Open-Meteo air quality.")
        pm_df = fetch_openmeteo_aq_pm25(LAT, LON, hours=HOURS_BACK)

    if pm_df.empty:
        raise RuntimeError("Failed to fetch any PM2.5 data. Check network / API availability.")

    # Fetch weather features (temperature, humidity, wind)
    weather_df = fetch_openmeteo_weather(LAT, LON, hours=HOURS_BACK)

    merged_df = merge_datasets(pm_df, weather_df)
    cleaned_df = clean_data(merged_df)
    cleaned_df = remove_outliers(cleaned_df)
    cleaned_df = detect_anomalies(cleaned_df)

    processed_path = "data/processed/pm25_merged.csv"
    cleaned_df.to_csv(processed_path, index=False)
    print(f"📁 Processed data saved to {processed_path}")

    # Train model
    train_model(cleaned_df, model_out="models/model.joblib")


if __name__ == "__main__":
    main()


No OpenAQ data found — falling back to Open-Meteo air quality.
🔗 Merging datasets...
🧹 Cleaning data...


/tmp/ipython-input-929650355.py:150: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  merged = pd.merge_asof(pm, w, on="timestamp", direction="nearest", tolerance=pd.Timedelta("1H"))
/tmp/ipython-input-929650355.py:165: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample("1H").mean()
/tmp/ipython-input-929650355.py:239: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = df[[col]].fillna(method="ffill").fillna(method="bfill")


🚫 Removing outliers...
⚠️ Detecting anomalies (IsolationForest)...
📁 Processed data saved to data/processed/pm25_merged.csv
🧠 Creating time, lag, and rolling features...
🚀 Training model (Gradient Boosting)...
✅ Validation MAE: 3.370
💾 Model saved to models/model.joblib
